In [2]:
from typing import List
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama
import smtplib
from email.message import EmailMessage

# --- TypedDict for input schema ---
class EmailInput(TypedDict):
    sender_email: str
    app_password: str
    recipient_email: str
    subject: str
    body: str


# --- Email sending function ---
def send_gmail_email_llm(data: dict) -> dict:
    """
    Sends an email using Gmail SMTP. Injects required auth fields if missing.
    """
    # Inject your actual Gmail credentials (ideally pulled from env vars or a secrets manager)
    data['sender_email'] = data.get('sender_email', 'your_email@gmail.com')
    data['app_password'] = data.get('app_password', 'your_gmail_app_password')
    data['recipient_email'] = data.get('recipient_email', data.get('to'))  # fallback

    # Build and send email
    msg = EmailMessage()
    msg['Subject'] = data['subject']
    msg['From'] = data['sender_email']
    msg['To'] = data['recipient_email']
    msg.set_content(data['body'])

    smtp_server = 'smtp.gmail.com'
    smtp_port = 587

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(data['sender_email'], data['app_password'])
            server.send_message(msg)
            print("Email sent successfully!")
            return {"status": "success", "message": "Email sent successfully!"}
    except Exception as e:
        print("Error:", e)
        return {"status": "error", "message": str(e)}


# --- User validation function ---
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """
    Validate user using historical addresses.

    Args:
        user_id: The user ID.
        addresses: List of previous addresses.
    """
    return True

# --- Bind functions to the LLM ---
llm = ChatOllama(
    model="llama3-groq-tool-use:8b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
).bind_tools([validate_user, send_gmail_email_llm])


In [3]:
result_validate = llm.invoke(
    "Could you validate user 123? Thet previously lived at "
    "123 Fake st in boston MA and 234 Pretend Boulevard in"
    "Houston TX."
)

result_validate.tool_calls


[{'name': 'validate_user',
  'args': {'addresses': ['123 Fake st, Boston MA',
    '234 Pretend Boulevard, Houston TX'],
   'user_id': 123},
  'id': 'f7fc9e3e-8494-4982-b03e-758dee900189',
  'type': 'tool_call'}]

In [15]:
result_email = llm.invoke(
    "Send an email to nattapat.7140@gmail.com saying 'Hi Best, hope you're doing well. Just checking in!' with subject 'Quick Hello'. "
    "Use sender email nattapat.0645@gmail.com and app password a0645402317."
)


print(result_email.tool_calls)
print("#########################")
print(result_email.content)

[{'name': 'send_gmail_email_llm', 'args': {'data': {'body': "Hi Best, hope you're doing well. Just checking in!", 'from': 'nattapat.0645@gmail.com', 'password': 'a0645402317', 'subject': 'Quick Hello', 'to': 'nattapat.7140@gmail.com'}}, 'id': 'c903ae92-5e9f-4e34-8b87-8ee6670174af', 'type': 'tool_call'}]
#########################



In [14]:
result_none_tools = llm.invoke(
    "who make you and give me briefly about you?"
)

print(result_none_tools.tool_calls)
print(result_none_tools.content)

[]
I was created by a team of developers who designed me to assist with various tasks such as validating user information, sending emails, and more. I'm here to help you with any questions or requests you have!


In [3]:
result_multi_tools = llm.invoke(
    "Could you validate user 123? Thet previously lived at "
    "123 Fake st in boston MA and 234 Pretend Boulevard in"
    "Houston TX."
    "Then"
    "Send an email to nattapat.7140@gmail.com saying 'Hi Best, hope you're doing well. Just checking in!' with subject from user validation result. "
    "Use sender email nattapat.0645@gmail.com and app password a0645402317."
)

print(result_multi_tools.tool_calls)
print(result_multi_tools.content)

[{'name': 'validate_user', 'args': {'addresses': ['123 Fake st in boston MA', '234 Pretend Boulevard inHouston TX'], 'user_id': 123}, 'id': 'e1d81b24-a8c5-476a-acd4-f8b52b63053c', 'type': 'tool_call'}]

